# Accelerated Database Recovery - Speed up Recovery
In this example you will learn how Accelerated Database Recovery will speed up recovery



## Step 1: Setup the database
Create a database larger enough to hold data and tlog without autogrow.

Depending on the speed of your server, creating the database and data could take several minutes.

**Note**: *For Linux installations the default path to use is /var/opt/mssql*

In [1]:
USE master
GO
DROP DATABASE IF EXISTS gocowboys
GO
CREATE DATABASE gocowboys
ON PRIMARY
(NAME = N'gocowboys_primary', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\gocowboys.mdf', SIZE = 5Gb , MAXSIZE = UNLIMITED, FILEGROWTH = 65536KB)
LOG ON 
(NAME = N'gocowboys_log', FILENAME = 'C:\Program Files\Microsoft SQL Server\MSSQL15.MSSQLSERVER\MSSQL\DATA\gocowboys_log.ldf', SIZE = 20Gb , MAXSIZE = UNLIMITED , FILEGROWTH = 65536KB)
GO
ALTER DATABASE gocowboys SET RECOVERY SIMPLE
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:32.172

## Step 2: Create the table
Create a table with a million rows so that when we delete them all recovery will take a long time

In [2]:
USE gocowboys
GO
DROP TABLE IF EXISTS howboutthemcowboys
GO
CREATE TABLE howboutthemcowboys (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 500000)
BEGIN
	INSERT INTO howboutthemcowboys VALUES (@x, 'Jason Witten returns in 2019')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:00:43.153

## Step 3: Delete all the rows in the table
Delete all the rows in the table in a user defined transaction

In [3]:
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = OFF
GO
-- Try to delete a bunch of rows
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

(500000 rows affected)

Total execution time: 00:00:28.489

## Step 4: Checkpoint the database, shutdown SQL Server WITH NOWAIT, and restart it
Checkpoint will flush any dirty pages but the transaction is not committed. Shutting down the server with NOWAIT will not attempt to roll back active transactions. Therefore, SQL Server has to run undo to rollback this transaction. 

Separate from this notebook, use a New Query in Azure Data Studio and run the following commands as a sysadmin. 

**Restart SQL Server and then examine the ERRORLOG to see how long recovery takes for the database. SQL Server must fully recover the database before you go to the next step. Because there is so much to recover this could take 10 minutes or more**

```sql
USE gocowboys
GO
CHECKPOINT
GO
SHUTDOWN WITH NOWAIT
GO
```


## Step 5: Create the table again for a second test
Use the same code as before to create the table with a million rows. But turn on Accelerated Database Recovery first.

You may first see a connection error first since you shutdown SQL Server in the previous step but the cell should retry the connection and run the T-SQL statements.

In [4]:
USE master
GO
ALTER DATABASE gocowboys SET ACCELERATED_DATABASE_RECOVERY = ON
GO
USE gocowboys
GO
DROP TABLE IF EXISTS howboutthemcowboys
GO
CREATE TABLE howboutthemcowboys (playerid int primary key clustered, playername char(7000) not null)
GO
SET NOCOUNT ON
GO
BEGIN TRAN
DECLARE @x int
SET @x = 0
WHILE (@x < 500000)
BEGIN
	INSERT INTO howboutthemcowboys VALUES (@x, 'Jason Witten returns in 2019')
	SET @x = @x + 1
END
COMMIT TRAN
GO
SET NOCOUNT OFF
GO
USE master
GO

: Msg 0, Level 20, State 0, Line 0
The connection is broken and recovery is not possible.  The connection is marked by the server as unrecoverable.  No attempt was made to restore the connection.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Commands completed successfully.

Total execution time: 00:01:13.750

## Step 6: Delete all the rows in the table in a transaction
Same code as before to delete all rows and not commit or rollback the transaction

In [5]:
USE gocowboys
GO
BEGIN TRAN
DELETE from howboutthemcowboys
GO

Commands completed successfully.

(1000000 rows affected)

Total execution time: 00:00:27.279

## Step 7:  Checkpoint the database, shutdown SQL Server WITH NOWAIT, and restart it with Accelerated Database Recovery
Checkpoint will flush any dirty pages but the transaction is not committed. Shutting down the server with NOWAIT will not attempt to roll back active transactions. Therefore, SQL Server has to run undo to rollback this transaction. 

Separate from this notebook, use a New Query in Azure Data Studio and run the following commands as a sysadmin. Then restart SQL Server to examine the ERRORLOG to see how recovery takes for the database. You should see that recovery is significantly faster than before.

```sql
USE gocowboys
GO
CHECKPOINT
GO
SHUTDOWN WITH NOWAIT
GO
```